In [31]:
import pandas as pd

df_jan = pd.read_parquet('fhv_tripdata_2021-01.parquet')
df_feb = pd.read_parquet('fhv_tripdata_2021-02.parquet')

In [32]:
df_jan.shape

(1154112, 7)

In [33]:
df_jan['duration'] = df_jan.dropOff_datetime - df_jan.pickup_datetime
df_feb['duration'] = df_feb.dropOff_datetime - df_feb.pickup_datetime

In [34]:
df_jan.duration = df_jan.duration.apply(lambda td: td.total_seconds() / 60)
df_feb.duration = df_feb.duration.apply(lambda td: td.total_seconds() / 60)

In [35]:
df_jan.duration.mean()

19.167224093791006

In [36]:
df_jan = df_jan[(df_jan.duration >= 1) & (df_jan.duration <= 60)]
df_feb = df_feb[(df_feb.duration >= 1) & (df_feb.duration <= 60)]

In [37]:
df_jan.shape

(1109826, 8)

In [38]:
df_feb.shape

(990113, 8)

In [39]:
1154112 - 1109826

44286

In [40]:
df_jan.PUlocationID.isnull().sum()

927008

In [41]:
(927008/1109826)*100

83.52732770722618

In [42]:
df_jan.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009,17.000000
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009,17.000000
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037,8.283333
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037,15.216667
5,B00037,2021-01-01 00:59:02,2021-01-01 01:08:05,NaN,71.0,None,B00037,9.050000


In [43]:
df_jan[['DOlocationID', 'PUlocationID']] = df_jan[['DOlocationID', 'PUlocationID']].fillna(-1)

In [44]:
df_feb[['DOlocationID', 'PUlocationID']] = df_feb[['DOlocationID', 'PUlocationID']].fillna(-1)

In [45]:
from sklearn.feature_extraction import DictVectorizer

dv = DictVectorizer()
train_dicts = df_jan[['DOlocationID', 'PUlocationID']].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_feb[['DOlocationID', 'PUlocationID']].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [46]:
X_train.shape

(1109826, 2)

In [47]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

y_train = df_jan.duration.values
y_val = df_feb.duration.values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

print(mean_squared_error(y_train, y_pred,  squared=False))

y_pred = lr.predict(X_val)

print(mean_squared_error(y_val, y_pred,  squared=False))

11.415432830521663
11.85822362355935
